# Processing available DGFP data: dp_dgfp-imp11subdist

## Getting necessary functions

In [1]:
import os
import glob
import pandas as pd

## Custom functions

In [6]:
def read_data(path, ext):
    files_list = glob.glob(path + '/*.' + ext)
    data = list()
    for file in files_list:
        file_name = os.path.split(file)[1][:-4]
        program = file.split("/")[-2]
        try:
            tmp = pd.read_csv(file, sep='\t', comment=' ')
        except pd.errors.EmptyDataError:
            # print(f"{file} has no columns to parse. Moving on")
            continue
        file_name_comp = file_name.split('-')
        tmp['Year'] = file_name_comp[0]
        tmp['Month'] = file_name_comp[1]
        tmp['District'] =file_name_comp[3]
        tmp['Division'] = file_name_comp[4]
        tmp['program'] = program
        data.append(tmp)
    return pd.concat(data, axis=0)

def read_data_dir(data_dirs, ext):
    data_dict= {}
    dirs = pd.read_csv(data_dirs, header=None, names=['dirs'])
    for folder in dirs['dirs']:
        # print(folder)
        data_dict[folder] = read_data(path=folder, ext=ext)
    return data_dict

def break_by_years(data_dict, start_year, end_year, year_var):
    years = [str(year) for year in range(start_year, end_year+1)]
    out_dict = {}
    for index, item in enumerate(years):
        out_dict[str(item)] = {}
        for key, value in data_dict.items():
            if str(item) in list(value[year_var].unique()):
                out_dict[str(item)][key] = value[value[year_var] == str(item)]
            else:
                pass
    return out_dict

def summarize_data(data_dict, index_vars):
    years_dict = {}
    for key, value in data_dict.items():
        years_dict[key] = {}
        for program, df in value.items():
            tmp_index = list(set(index_vars).intersection(set(df.columns)))
            df = df.set_index(tmp_index)
            df = df[list(df.dtypes[df.dtypes == 'float64'].index)]
            try:
                df = df.groupby(['thana']).sum()
            except KeyError:
                df = df.groupby(['thana_Name']).sum()
            df.columns = [var + "_" + program.split("/")[-1] for var in df.columns]
            years_dict[key][program] = df
    return years_dict 

def merge_by_years(data_dict):
    out = {}
    for year, data in data_dict.items():
        out[year] = pd.concat([v for k,v in data.items()], axis=1, sort=True, join='outer',)
    return out

def write_out(data_dict):
    out = {}
    for year, data in data_dict.items():
        out[year] = pd.concat([v for k,v in data.items()], axis=1, sort=True, join='outer',)
    return out

## DGFP data preprocessing

In [7]:
DATA_DIRS = '../../output/dgfp/data_dirs.txt'
dgfp_imp11subdist = read_data_dir(data_dirs=DATA_DIRS, ext='csv')

In [31]:
def dgfp_geos(data_dict, geo_vars):
    out = {}
    for key, value in data_dict.items():
        subset = list(set(geo_vars).intersection(set(value.columns)))
        tmp = value[subset]
        tmp = tmp.drop_duplicates()
        out[key] = tmp
    return out
        
t = dgfp_geos(data_dict=dgfp_imp11subdist, geo_vars = ['thana_Name', 'thana', 'District', 'Division', 'program'])
for k,v in t.items():
    print("#"*100)
    print(k)
    print(v.shape)
    display(v.head())
t['../../data/dgfp/data/importance11_subdistricts/thana_process'].to_csv("../../dgfp_geo.csv")

####################################################################################################
../../data/dgfp/data/importance11_subdistricts/ngothana_process
(381, 4)


,thana,District,Division,program
0,Panchgarh,0103,07,ngothana_process
1,Atowari,0103,07,ngothana_process
2,Debiganj,0103,07,ngothana_process
0,Moheshpur,0204,02,ngothana_process
1,Jhenaidaha,0204,02,ngothana_process


####################################################################################################
../../data/dgfp/data/importance11_subdistricts/mchgo_thanaprocess
(488, 4)


,thana_Name,District,Division,program
0,Mymensingh,0415,08,mchgo_thanaprocess
1,Trisal,0415,08,mchgo_thanaprocess
2,Fulbaria,0415,08,mchgo_thanaprocess
3,Bhaluka,0415,08,mchgo_thanaprocess
4,Muktagacha,0415,08,mchgo_thanaprocess


####################################################################################################
../../data/dgfp/data/importance11_subdistricts/thana_process
(907, 4)


,thana,District,Division,program
0,Boda,0103,07,thana_process
1,Panchgarh,0103,07,thana_process
2,Tetulia,0103,07,thana_process
3,Atowari,0103,07,thana_process
4,Debiganj,0103,07,thana_process


In [12]:
for key,value in dgfp_imp11subdist.items():
    print(key)
    for year,df in value.items():
        print(year)

../../data/dgfp/data/importance11_subdistricts/ngothana_process
thana
ECouple
Pill
Condom
Injectable
IUD
Implant
PerMale
PerFemale
Tpermanent
GrandTotal
CAR
Unnamed: 12
Year
Month
District
Division
program
../../data/dgfp/data/importance11_subdistricts/mchgo_thanaprocess
thana_Name
MCH37
MCH38
MCH39
MCH40
MCH41
MCH42
MCH43
MCH44
MCH45
MCH46
MCH47
MCH48
MCH49
MCH50
MCH51
MCH52
MCH53
MCH54
MCH55
MCH56
MCH57
MCH58
MCH59
MCH60
MCH61
MCH62
MCH63
MCH64
MCH65
MCH66
MCH67
MCH68
MCH69
MCH70
MCH71
MCH72
MCH73
MCH74
MCH75
MCH76
MCH77
MCH78
MCH79
MCH80
MCH81
MCH82
MCH83
MCH84
MCH85
MCH86
MCH87
MCH88
MCH89
MCH90
MCH91
MCH92
MCH93
MCH94
MCH95
MCH96
MCH97
MCH98
MCH99
MCH100
MCH101
MCH102
MCH103
MCH104
MCH105
MCH106
MCH107
Year
Month
District
Division
program
../../data/dgfp/data/importance11_subdistricts/thana_process
thana
ECouple
Pill
Condom
Injectable
IUD
Implant
PerMale
PerFemale
Tpermanent
GrandTotal
CAR
Unnamed: 12
Year
Month
District
Division
program


In [4]:
dgfp_imp11subdist_years = break_by_years(data_dict=dgfp_imp11subdist, start_year=2008, end_year=2018, year_var='Year')
dgfp_imp11subdist_data = summarize_data(data_dict = dgfp_imp11subdist_years, index_vars=['thana_Name', 'thana', 'Year', 'District', 'Division', 'program'])
data = merge_by_years(dgfp_imp11subdist_data)    

In [305]:
for key, value in data.items():
    display(value.head())

,ECouple_ngothana_process,Pill_ngothana_process,Condom_ngothana_process,Injectable_ngothana_process,IUD_ngothana_process,Implant_ngothana_process,PerMale_ngothana_process,PerFemale_ngothana_process,Tpermanent_ngothana_process,GrandTotal_ngothana_process,...,Condom_thana_process,Injectable_thana_process,IUD_thana_process,Implant_thana_process,PerMale_thana_process,PerFemale_thana_process,Tpermanent_thana_process,GrandTotal_thana_process,CAR_thana_process,Unnamed: 12_thana_process
Abhaynagar,47669.0,10137.0,7050.0,13879.0,2851.0,2302.0,1284.0,1685.0,2969.0,39188.0,...,34124.0,125889.0,22351.0,9204.0,22882.0,50632.0,73514.0,487779.0,909.8598,0.0
Adamdighi,19830.0,9670.0,1092.0,2059.0,439.0,378.0,270.0,1748.0,2018.0,15656.0,...,30741.0,48056.0,12299.0,7432.0,11667.0,57734.0,69401.0,470194.0,1086.0151,0.0
Adarsho,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0000,0.0
Aditmari,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,7980.0,62619.0,5427.0,15282.0,14910.0,62943.0,77853.0,392588.0,852.6573,0.0
Agailjhara,11819.0,5664.0,428.0,608.0,162.0,78.0,24.0,2198.0,2222.0,9162.0,...,15141.0,22932.0,13186.0,3449.0,1905.0,45235.0,47140.0,241045.0,877.3506,0.0


,ECouple_ngothana_process,Pill_ngothana_process,Condom_ngothana_process,Injectable_ngothana_process,IUD_ngothana_process,Implant_ngothana_process,PerMale_ngothana_process,PerFemale_ngothana_process,Tpermanent_ngothana_process,GrandTotal_ngothana_process,...,Condom_thana_process,Injectable_thana_process,IUD_thana_process,Implant_thana_process,PerMale_thana_process,PerFemale_thana_process,Tpermanent_thana_process,GrandTotal_thana_process,CAR_thana_process,Unnamed: 12_thana_process
Abhaynagar,57090.0,12213.0,9666.0,18531.0,3772.0,2886.0,1760.0,2207.0,3967.0,51035.0,...,38238.0,142886.0,25043.0,9652.0,25861.0,56371.0,82232.0,511682.0,940.5619,0.0
Adamdighi,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,26826.0,47277.0,11797.0,6134.0,11736.0,53215.0,64951.0,413542.0,947.5483,0.0
Adarsho,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0000,0.0
Aditmari,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,8585.0,83519.0,6466.0,18175.0,16544.0,64520.0,81064.0,418711.0,905.7718,0.0
Agailjhara,17880.0,7993.0,648.0,1066.0,312.0,84.0,37.0,3304.0,3341.0,13444.0,...,15450.0,28677.0,12722.0,3400.0,2444.0,46290.0,48734.0,244821.0,882.5708,0.0


,ECouple_ngothana_process,Pill_ngothana_process,Condom_ngothana_process,Injectable_ngothana_process,IUD_ngothana_process,Implant_ngothana_process,PerMale_ngothana_process,PerFemale_ngothana_process,Tpermanent_ngothana_process,GrandTotal_ngothana_process,...,Condom_thana_process,Injectable_thana_process,IUD_thana_process,Implant_thana_process,PerMale_thana_process,PerFemale_thana_process,Tpermanent_thana_process,GrandTotal_thana_process,CAR_thana_process,Unnamed: 12_thana_process
Abhaynagar,50705.0,12026.0,9768.0,12999.0,1692.0,1435.0,1307.0,2966.0,4273.0,42193.0,...,44131.0,146445.0,20712.0,6473.0,26338.0,59016.0,85354.0,516874.0,950.9027,0.0
Adamdighi,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,30279.0,51210.0,13689.0,4834.0,15355.0,58686.0,74041.0,450987.0,1026.7137,0.0
Adarsho,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0000,0.0
Aditmari,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,10362.0,94028.0,6101.0,14713.0,22473.0,61859.0,84332.0,430744.0,913.0970,0.0
Agailjhara,18273.0,8356.0,861.0,1107.0,309.0,93.0,58.0,2959.0,3017.0,13743.0,...,16160.0,28071.0,10517.0,3458.0,3004.0,37098.0,40102.0,235028.0,868.5212,0.0


,ECouple_ngothana_process,Pill_ngothana_process,Condom_ngothana_process,Injectable_ngothana_process,IUD_ngothana_process,Implant_ngothana_process,PerMale_ngothana_process,PerFemale_ngothana_process,Tpermanent_ngothana_process,GrandTotal_ngothana_process,...,Condom_thana_process,Injectable_thana_process,IUD_thana_process,Implant_thana_process,PerMale_thana_process,PerFemale_thana_process,Tpermanent_thana_process,GrandTotal_thana_process,CAR_thana_process,Unnamed: 12_thana_process
Abhaynagar,41828.0,10007.0,8613.0,10467.0,1232.0,1002.0,1202.0,2771.0,3973.0,35294.0,...,50462.0,166712.0,22449.0,11083.0,30765.0,71974.0,102739.0,585825.0,1062.3264,0.0
Adamdighi,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,29458.0,46965.0,13836.0,8187.0,15973.0,57336.0,73309.0,427729.0,967.5146,0.0
Adarsho,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0000,0.0
Aditmari,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,11269.0,103606.0,7497.0,17901.0,31139.0,63377.0,94516.0,452116.0,943.8363,0.0
Agailjhara,18504.0,8444.0,921.0,1107.0,315.0,174.0,62.0,3033.0,3095.0,14056.0,...,15952.0,31221.0,10925.0,5263.0,3596.0,38369.0,41965.0,245529.0,906.5910,0.0


,ECouple_ngothana_process,Pill_ngothana_process,Condom_ngothana_process,Injectable_ngothana_process,IUD_ngothana_process,Implant_ngothana_process,PerMale_ngothana_process,PerFemale_ngothana_process,Tpermanent_ngothana_process,GrandTotal_ngothana_process,...,Condom_thana_process,Injectable_thana_process,IUD_thana_process,Implant_thana_process,PerMale_thana_process,PerFemale_thana_process,Tpermanent_thana_process,GrandTotal_thana_process,CAR_thana_process,Unnamed: 12_thana_process
Abhaynagar,46270.0,10747.0,9506.0,11744.0,1185.0,1079.0,1573.0,3266.0,4839.0,39100.0,...,49603.0,157820.0,21051.0,14639.0,30330.0,72583.0,102913.0,559926.0,1001.2778,0.0
Adamdighi,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,33227.0,51276.0,16028.0,12016.0,20756.0,64480.0,85236.0,476117.0,1066.3278,0.0
Adarsho,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0000,0.0
Aditmari,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,12663.0,110272.0,9549.0,21191.0,41371.0,65400.0,106771.0,477707.0,981.4929,0.0
Agailjhara,12596.0,5558.0,703.0,705.0,215.0,200.0,42.0,2085.0,2127.0,9508.0,...,16694.0,32606.0,11137.0,6640.0,4190.0,39696.0,43886.0,252818.0,910.3286,0.0


...

,ECouple_ngothana_process,Pill_ngothana_process,Condom_ngothana_process,Injectable_ngothana_process,IUD_ngothana_process,Implant_ngothana_process,PerMale_ngothana_process,PerFemale_ngothana_process,Tpermanent_ngothana_process,GrandTotal_ngothana_process,...,Condom_thana_process,Injectable_thana_process,IUD_thana_process,Implant_thana_process,PerMale_thana_process,PerFemale_thana_process,Tpermanent_thana_process,GrandTotal_thana_process,CAR_thana_process,Unnamed: 12_thana_process
Abhaynagar,59385.0,13929.0,7896.0,14432.0,1388.0,1194.0,1327.0,3080.0,4407.0,43246.0,...,45851.0,144354.0,18225.0,19034.0,27882.0,72625.0,100507.0,527433.0,931.2207,0.0
Adamdighi,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,35907.0,51293.0,15618.0,14496.0,21294.0,64028.0,85322.0,480552.0,1056.0948,0.0
Adarsho,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0000,0.0
Aditmari,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,13608.0,106518.0,9210.0,32872.0,40635.0,54612.0,95247.0,457032.0,959.9567,0.0
Agailjhara,12503.0,5508.0,701.0,847.0,121.0,270.0,33.0,2128.0,2161.0,9608.0,...,17509.0,30984.0,9928.0,10377.0,4832.0,40191.0,45023.0,254789.0,891.2533,0.0


,ECouple_ngothana_process,Pill_ngothana_process,Condom_ngothana_process,Injectable_ngothana_process,IUD_ngothana_process,Implant_ngothana_process,PerMale_ngothana_process,PerFemale_ngothana_process,Tpermanent_ngothana_process,GrandTotal_ngothana_process,...,Condom_thana_process,Injectable_thana_process,IUD_thana_process,Implant_thana_process,PerMale_thana_process,PerFemale_thana_process,Tpermanent_thana_process,GrandTotal_thana_process,CAR_thana_process,Unnamed: 12_thana_process
Abhaynagar,54677.0,12458.0,6859.0,13716.0,1514.0,1253.0,1367.0,3039.0,4406.0,40206.0,...,48148.0,146378.0,18768.0,23140.0,28975.0,75938.0,104913.0,543905.0,948.1124,0.0
Adamdighi,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,35553.0,49742.0,14744.0,17373.0,20429.0,60887.0,81316.0,454105.0,990.5567,0.0
Adarsho,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0000,0.0
Aditmari,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,13107.0,109044.0,9356.0,42464.0,41232.0,55701.0,96933.0,466061.0,965.9741,0.0
Agailjhara,10288.0,4455.0,576.0,623.0,117.0,290.0,41.0,1644.0,1685.0,7746.0,...,18563.0,32215.0,10230.0,11980.0,4939.0,40900.0,45839.0,263276.0,911.9599,0.0


,ECouple_ngothana_process,Pill_ngothana_process,Condom_ngothana_process,Injectable_ngothana_process,IUD_ngothana_process,Implant_ngothana_process,PerMale_ngothana_process,PerFemale_ngothana_process,Tpermanent_ngothana_process,GrandTotal_ngothana_process,...,Condom_thana_process,Injectable_thana_process,IUD_thana_process,Implant_thana_process,PerMale_thana_process,PerFemale_thana_process,Tpermanent_thana_process,GrandTotal_thana_process,CAR_thana_process,Unnamed: 12_thana_process
Abhaynagar,75654.0,18605.0,9185.0,19586.0,2019.0,2195.0,2459.0,5118.0,7577.0,59167.0,...,55887.0,137311.0,15468.0,23601.0,27222.0,58139.0,85361.0,522172.0,962.4946,0.0
Adamdighi,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,39062.0,51644.0,15047.0,19887.0,21274.0,64566.0,85840.0,477962.0,1059.3481,0.0
Adarsho,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0000,0.0
Aditmari,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,14125.0,106299.0,8739.0,41206.0,38439.0,49431.0,87870.0,449637.0,944.6572,0.0
Agailjhara,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,18330.0,31489.0,9282.0,13687.0,4697.0,38647.0,43344.0,258401.0,924.2011,0.0


,ECouple_ngothana_process,Pill_ngothana_process,Condom_ngothana_process,Injectable_ngothana_process,IUD_ngothana_process,Implant_ngothana_process,PerMale_ngothana_process,PerFemale_ngothana_process,Tpermanent_ngothana_process,GrandTotal_ngothana_process,...,Condom_thana_process,Injectable_thana_process,IUD_thana_process,Implant_thana_process,PerMale_thana_process,PerFemale_thana_process,Tpermanent_thana_process,GrandTotal_thana_process,CAR_thana_process,Unnamed: 12_thana_process
Abhaynagar,81266.0,20073.0,9911.0,21177.0,2286.0,2742.0,2875.0,5829.0,8704.0,64893.0,...,57754.0,138655.0,16642.0,27903.0,27723.0,58640.0,86363.0,535072.0,981.1204,0.0
Adamdighi,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,39875.0,52129.0,15992.0,22523.0,21564.0,65980.0,87544.0,484292.0,1068.6110,0.0
Adarsho,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0000,0.0
Aditmari,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,15117.0,108509.0,12058.0,44219.0,38265.0,50491.0,88756.0,460349.0,958.5767,0.0
Agailjhara,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,18309.0,32798.0,9229.0,16717.0,4775.0,39194.0,43969.0,260914.0,929.2787,0.0


,ECouple_ngothana_process,Pill_ngothana_process,Condom_ngothana_process,Injectable_ngothana_process,IUD_ngothana_process,Implant_ngothana_process,PerMale_ngothana_process,PerFemale_ngothana_process,Tpermanent_ngothana_process,GrandTotal_ngothana_process,...,Condom_thana_process,Injectable_thana_process,IUD_thana_process,Implant_thana_process,PerMale_thana_process,PerFemale_thana_process,Tpermanent_thana_process,GrandTotal_thana_process,CAR_thana_process,Unnamed: 12_thana_process
Abhaynagar,71094.0,17509.0,9136.0,16448.0,1959.0,3140.0,2448.0,5062.0,7510.0,55702.0,...,43304.0,101068.0,12718.0,23447.0,20734.0,44130.0,64864.0,401665.0,729.6482,0.0
Adamdighi,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,28256.0,36393.0,11677.0,17446.0,15295.0,46329.0,61624.0,340377.0,748.0949,0.0
Adarsho,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0000,0.0
Aditmari,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,11895.0,83769.0,10609.0,35705.0,28929.0,39337.0,68266.0,355807.0,732.5674,0.0
Agailjhara,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,13832.0,25786.0,7144.0,13608.0,3610.0,29736.0,33346.0,197163.0,696.5413,0.0
